<a href="https://colab.research.google.com/github/h-gokul/ReinforcementLearningBasics/blob/master/DQN_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q Learning networks 
  Every reward and state achieved by a agent in the environement is the *function* of the action performed by the agent in that state of the environment. This function was usually mapped with a Q table that evaluates the Q value, regularly updated - thereby learnt, for every sequence of actions performed. - This is Q learning.

  In Reinforcement Learning, the idea of implementing Q learning and storing them in a Q table can be optimal and viable for small problems. But desigining a LUT for a complex environment and agent requires multitudes of data to be stored in 'actionspace x observationspace' dimensions. Thus, by replacing a Q table with a neural network to obtain this *function* for any action performed, we manage to save data and improve accuracy.   

 The notebook explores a DQN version of the  player vs food vs enemy  Blob game which was previously done in Q-table method

In [ ]:
import numpy as np
import keras.backend.tensorflow_backend as backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import tensorflow as tf
from collections import deque
import time
import random
from tqdm import tqdm
import os
from PIL import Image
import cv2


DISCOUNT = 0.99
REPLAY_MEMORY_SIZE = 50_000  # How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 1_000  # Minimum number of steps in a memory to start training
MINIBATCH_SIZE = 64  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)
MODEL_NAME = '2x256'
MIN_REWARD = -200  # For model save
MEMORY_FRACTION = 0.20

# Environment settings
EPISODES = 20_000

# Exploration settings
epsilon = 1  # not a constant, going to be decayed
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001

#  Stats settings
AGGREGATE_STATS_EVERY = 50  # episodes
SHOW_PREVIEW = False

Using TensorFlow backend.


In every episode, every step we take, we want to update Q values, but we also are trying to predict from our model.
Especially initially, our model is starting off as random, and it's being updated every single step, per every single episode.
What ensues here are massive fluctuations that are super confusing to our model. 
This is why we almost always train neural networks with batches (that and the time-savings).
One way this is solved is through a concept of memory replay, whereby we actually have two models. 'model' is the actual model that we are training.
'target_model' is updated every n episodes(a parameter to be decided)
Eventually, we converge the two models so they are the same, but we want the model that we query for future Q values to be more stable than the model that we're actively fitting every single step.
###### Replay Memory 
- This concept is implemented to avoid fluctuations that occur due to extensive number of .fit() and .predict() that happen every iteration every episode with single sample training.
-  Thus, we can maintain  a memory for our agent, that remembers the previous say 1000 actions and train as minibatches


# The DQN Agent routine

We initialise a DQN agent, which uses two models to  gradually update a final 'model' in minibatches. The minibatch from the replay memory already has a set of states, actions and the corresponding rewards stored in it. The Model is a 2 layer 2D Convnet model. It's input will be the observation space - (say 10x10x9) and its output will be actions (9 actions in this case). Th model is trained as follows

- If the replay memory is large enough, - begin training.
#### Process the Q values for every action in a batch  
- Random sample a minibatch (size 64x5) from the replay memory (REPLAYMEMORY_SIZE x 5) : 64 = MINIBATCH_SIZE; 5 = len(currentstates,action,reward,newstates,done). Current states and new states are images.
- Obtain list of current states from the batch, perform the prediction in 'model' to obtain list of current Q values.
- Obtain list of new states from the batch , perform the prediction in 'target model' to obtain list of future Q values (this is not new Q values. Remember new Q can be found only by the optimization learning function).
- if the routine is not done, calculate maxfutureQ = max of the list (future Q values). Using maxfutureq and the reward, find the newQ for that action.  Else, where it will be 'done', there is not maxfutureQ so newQ=reward for that action.
- For a given action, update the corresponding Q value from the currentqlist with the new q values.  
- Repeat this process through the whole batch. Now we have  a batch with updated Q values for every action performed resulted from the previous model prediction
#### Train the model.
-   'X' - Current state images  'y' -current q values after action-wise updation 
 
 
The 'model' is trained with X and Y for every iteration while 'target model' obtains the weights of 'model' 'UPDATE_TARGET_EVERY' iterations.



In [ ]:
class DQNAgent:
    
    def __init__(self):

        # main model  # gets trained every step
        self.model = self.create_model()

        # Target model this is what we .predict against every step
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE) # This memory variable can be dynamically extended from both sides
        self.tensorboard = ModifiedTensorBoard(log_dir=f"logs/{MODEL_NAME}-{int(time.time())}") # 
        self.target_update_counter = 0 

    
    # A simple Conv2D 2 layer neural net    
    def create_model(self):
        model = Sequential()

        model.add(Conv2D(256, (3, 3), input_shape=env.OBSERVATION_SPACE_VALUES))  # OBSERVATION_SPACE_VALUES = (10, 10, 3) a 10x10 RGB image.
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        model.add(Conv2D(256, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
        model.add(Dense(64))

        model.add(Dense(env.ACTION_SPACE_SIZE, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=['accuracy'])
        return model

     # Updates the replay memory, with the below values.
    # (observation space (current states), action, reward, new observation space(new states), done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    
    # to obtain Q values, which is to perform a .predict() 
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/255)[0] # reshape is done for shape compatibility.

    def train(self, terminal_state, step):
        
        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        # Obtain a minibatch of size MINIBATCH_SIZE from the replay_memory
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        # Obtain current states from the 'minbatch' 
        current_states = np.array([transition[0] for transition in minibatch])/255  # transistion[0] is the index of current states   
        current_qs_list = self.model.predict(current_states) # q value is obtained by a model. predict operation

        # Obtain new states from the 'minbatch' 
        new_current_states = np.array([transition[3] for transition in minibatch])/255  # transistion[3] is the index of current states   
        future_qs_list = self.target_model.predict(new_current_states) # q value is obtained by a model. predict operation

        ## Preparing the X and y to train the model
        X = []
        y = []
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)


        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X)/255, np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False, callbacks=[self.tensorboard] if terminal_state else None)

        if terminal_state:
          self.target_update_counter+=1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0
        

In [ ]:
# Since the .fit creates a log file every time it is called, we use the tensorboard class to skip unnecessary logging during this extensive training program.

class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)
    

# The participants

Blob components can be a food, player or an enemy. It can move in 9 directions. i.e 9 actions.

In [ ]:
class Blob:
    def __init__(self, size):
        self.size = size
        self.x = np.random.randint(0, size)
        self.y = np.random.randint(0, size)

    def __str__(self):
        return f"Blob ({self.x}, {self.y})"

    def __sub__(self, other):
        return (self.x-other.x, self.y-other.y)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def action(self, choice):
        '''
        Gives us 9 total movement options. (0,1,2,3,4,5,6,7,8)
        '''
        if choice == 0:
            self.move(x=1, y=1)
        elif choice == 1:
            self.move(x=-1, y=-1)
        elif choice == 2:
            self.move(x=-1, y=1)
        elif choice == 3:
            self.move(x=1, y=-1)

        elif choice == 4:
            self.move(x=1, y=0)
        elif choice == 5:
            self.move(x=-1, y=0)

        elif choice == 6:
            self.move(x=0, y=1)
        elif choice == 7:
            self.move(x=0, y=-1)

        elif choice == 8:
            self.move(x=0, y=0)

    def move(self, x=False, y=False):

        # If no value for x, move randomly (In case of food and enemy)
        if not x:
            self.x += np.random.randint(-1, 2)
        else:
            self.x += x

        # If no value for y, move randomly (In case of food and enemy)
        if not y:
            self.y += np.random.randint(-1, 2)
        else:
            self.y += y

        # If we are out of bounds, fix!
        if self.x < 0:
            self.x = 0
        elif self.x > self.size-1:
            self.x = self.size-1
        if self.y < 0:
            self.y = 0
        elif self.y > self.size-1:
            self.y = self.size-1
          

# The Environment

In [ ]:
class BlobEnv:
    SIZE = 10
    RETURN_IMAGES = True
    MOVE_PENALTY = 1
    ENEMY_PENALTY = 300
    FOOD_REWARD = 25
    OBSERVATION_SPACE_VALUES = (SIZE, SIZE, 3)  # 4
    ACTION_SPACE_SIZE = 9
    PLAYER_N = 1  # player key in dict
    FOOD_N = 2  # food key in dict
    ENEMY_N = 3  # enemy key in dict
    # the dict! (colors)
    d = {1: (255, 175, 0),
         2: (0, 255, 0),
         3: (0, 0, 255)}

    def reset(self):
        # defining the food and player
        self.player = Blob(self.SIZE)
        self.food = Blob(self.SIZE)
        while self.food == self.player:
            self.food = Blob(self.SIZE) # when food$ player are in same point, re call food
        
        self.enemy = Blob(self.SIZE)
        while self.enemy == self.player or self.enemy == self.food: # when food$ player are in same point, re call food
            self.enemy = Blob(self.SIZE)

        self.episode_step = 0

        if self.RETURN_IMAGES:
            observation = np.array(self.get_image())
        else:
            observation = (self.player-self.food) + (self.player-self.enemy)
        return observation

    def step(self, action): # When every step is proceeded
        self.episode_step += 1 # increment an episode step
        self.player.action(action) # player perfoms an action

        #### MAYBE ###
        #enemy.move()
        #food.move()
        ##############

        if self.RETURN_IMAGES: # image of the participant location is returned at end
            new_observation = np.array(self.get_image())
        else: 
            new_observation = (self.player-self.food) + (self.player-self.enemy)

        ## assigning rewards
        if self.player == self.enemy:
            reward = -self.ENEMY_PENALTY
        elif self.player == self.food:
            reward = self.FOOD_REWARD
        else:
            reward = -self.MOVE_PENALTY

        done = False
        if reward == self.FOOD_REWARD or reward == -self.ENEMY_PENALTY or self.episode_step >= 200:
            done = True

        return new_observation, reward, done


    def render(self):
        img = self.get_image()
        img = img.resize((300, 300))  # resizing so we can see our agent in all its glory.
        cv2.imshow("image", np.array(img))  # show it!
        cv2.waitKey(1)

    # FOR CNN #
    def get_image(self):
        env = np.zeros((self.SIZE, self.SIZE, 3), dtype=np.uint8)  # starts an rbg of our size
        env[self.food.x][self.food.y] = self.d[self.FOOD_N]  # sets the food location tile to green color
        env[self.enemy.x][self.enemy.y] = self.d[self.ENEMY_N]  # sets the enemy location to red
        env[self.player.x][self.player.y] = self.d[self.PLAYER_N]  # sets the player tile to blue
        img = Image.fromarray(env, 'RGB')  # reading to rgb. Apparently. Even tho color definitions are bgr. ???
        return img
      


In [ ]:
env = BlobEnv()

# For stats
ep_rewards = [-200]

# For more repetitive results
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

# Memory fraction, used mostly when trai8ning multiple agents
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=MEMORY_FRACTION)
#backend.set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))

# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')



# The process routine 

In [ ]:
agent = DQNAgent()

# Iterate over episodes
for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):

    # Update tensorboard step every episode
    agent.tensorboard.step = episode

    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1

    # Reset environment and get initial state
    current_state = env.reset() # rest- >  randomly place the food, player and target
    # print(current_state.shape)

    # Reset flag and start iterating until episode ends
    done = False
    while not done:

        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, env.ACTION_SPACE_SIZE)

        new_state, reward, done = env.step(action)

        # Transform new continous state to new discrete state and count reward
        episode_reward += reward

        if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:
            env.render()

        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)

        current_state = new_state
        step += 1

    # Append episode reward to a list and log stats (every given number of episodes)
    ep_rewards.append(episode_reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

        # Save model, but only when min reward is greater or equal a set value
        if min_reward >= MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')

    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

  0%|          | 0/20000 [00:00<?, ?episodes/s]

(10, 10, 3)


AttributeError: ignored